<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия № 2
</center>
Автор материала: Юрий Исаков и Юрий Кашницкий. Материал распространяется на условиях лицензии [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/). Можно использовать в любых целях (редактировать, поправлять и брать за основу), кроме коммерческих, но с обязательным упоминанием автора материала.

# <center>Тема 4. Линейные модели классификации и регрессии
## <center>  Практика. Идентификация пользователя с помощью логистической регрессии

Тут мы воспроизведем парочку бенчмарков нашего соревнования и вдохновимся побить третий бенчмарк, а также остальных участников. Веб-формы для отправки ответов тут не будет, ориентир – [leaderboard](https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/leaderboard) соревнования.

In [34]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
sklearn.__version__

'0.19.0'

### 1. Загрузка и преобразование данных
Зарегистрируйтесь на [Kaggle](www.kaggle.com), если вы не сделали этого раньше, зайдите на [страницу](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования и скачайте данные. Первым делом загрузим обучающую и тестовую выборки и посмотрим на данные.

In [35]:
# загрузим обучающую и тестовую выборки
train_df = pd.read_csv('../../data/train_sessions.csv',
                       index_col='session_id')
test_df = pd.read_csv('../../data/test_sessions.csv',
                      index_col='session_id')

# приведем колонки time1, ..., time10 к временному формату
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# отсортируем данные по времени
train_df = train_df.sort_values(by='time1')

In [46]:
# посмотрим на заголовок обучающей выборки
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55,2013-01-12 08:05:57,0,NaT,0,NaT,0,NaT,...,NaT,0,NaT,0,NaT,0,NaT,0,NaT,0
54843,56,2013-01-12 08:37:23,55,2013-01-12 08:37:23,56,2013-01-12 09:07:07,55,2013-01-12 09:07:09,0,NaT,...,NaT,0,NaT,0,NaT,0,NaT,0,NaT,0
77292,946,2013-01-12 08:50:13,946,2013-01-12 08:50:14,951,2013-01-12 08:50:15,946,2013-01-12 08:50:15,946,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948,2013-01-12 08:50:16,784,2013-01-12 08:50:16,949,2013-01-12 08:50:17,946,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948,2013-01-12 08:50:17,949,2013-01-12 08:50:18,948,2013-01-12 08:50:18,945,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947,2013-01-12 08:50:19,945,2013-01-12 08:50:19,946,2013-01-12 08:50:19,946,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950,2013-01-12 08:50:20,948,2013-01-12 08:50:20,947,2013-01-12 08:50:21,950,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946,2013-01-12 08:50:21,951,2013-01-12 08:50:22,946,2013-01-12 08:50:22,947,2013-01-12 08:50:22,0


В обучающей выборке содержатся следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - target – целевая переменная, 1 для сессий Элис, 0 для сессий других пользователей
    
Сессии пользователей выделены таким образом, что они не могут быть длиннее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд либо когда сессия заняла по времени более 30 минут.

В таблице встречаются пропущенные значения, это значит, что сессия состоит менее, чем из 10 сайтов. Заменим пропущенные значения нулями и приведем признаки к целому типу. Также загрузим словарь сайтов и посмотрим, как он выглядит:

In [38]:
# приведем колонки site1, ..., site10 к целочисленному формату и заменим пропуски нулями
sites = ['site%s' % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')

# загрузим словарик сайтов
with open(r"../../data/site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

# датафрейм словарика сайтов
sites_dict_df = pd.DataFrame(list(site_dict.keys()), 
                          index=list(site_dict.values()), 
                          columns=['site'])
print(u'всего сайтов:', sites_dict_df.shape[0])
sites_dict_df.head()

всего сайтов: 48371


,site
34555,i1-js-14-3-01-10005-734090045-i.init.cedexis-r...
8866,tahiti.oracle.com
25771,www.polymtl.ca
6023,www.mindat.org
12183,hub.loginradius.com


Выделим целевую переменную и объединим выборки, чтобы вместе привести их к разреженному формату.

In [39]:
# наша целевая переменная
y_train = train_df['target']

# объединенная таблица исходных данных
full_df = pd.concat([train_df.drop('target', axis=1), test_df])

# индекс, по которому будем отделять обучающую выборку от тестовой
idx_split = train_df.shape[0]

Для самой первой модели будем использовать только посещенные сайты в сессии (но не будем обращать внимание на временные признаки). За таким выбором данных для модели стоит такая идея:  *у Элис есть свои излюбленные сайты, и чем чаще вы видим эти сайты в сессии, тем выше вероятность, что это сессия Элис и наоборот.*

Подготовим данные, из всей таблицы выберем только признаки `site1, site2, ... , site10`. Напомним, что пропущенные значения заменены нулем. Вот как выглядят первые строки таблицы:

In [40]:
# табличка с индексами посещенных сайтов в сессии
full_sites = full_df[sites]
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947


Сессии представляют собой последовательность индексов сайтов и данные в таком виде неудобны для линейных методов. В соответствии с нашей гипотезой (у Элис есть излюбленные сайты) надо преобразовать эту таблицу таким образом, чтобы каждому возможному сайту соответствовал свой отдельный признак (колонка), а его значение равнялось бы количеству посещений этого сайта в сессии. Это делается в две строчки:

In [41]:
# последовательность с индексами
sites_flatten = full_sites.values.flatten()

# искомая матрица
full_sites_sparse = csr_matrix(([1] * sites_flatten.shape[0],
                                sites_flatten,
                                range(0, sites_flatten.shape[0] + 10, 10)))[:, 1:]

In [77]:
X_train_sparse = full_sites_sparse[:idx_split]
X_test_sparse = full_sites_sparse[idx_split:]
X_train_sparse.shape, X_test_sparse.shape

((253561, 48371), (82797, 48371))

In [43]:
vectorizer = TfidfVectorizer(vocabulary=[str(index) for index in sites_dict_df.index])
corpus = train_df.drop('target', axis=1)[sites].apply(lambda row: ' '.join([str(index) for index in row.values if index != 0]), axis=1).values
X_train_sparse = vectorizer.fit_transform(corpus)

In [48]:
vectorizer = TfidfVectorizer(vocabulary=[str(index) for index in sites_dict_df.index])
corpus = test_df[sites].apply(lambda row: ' '.join([str(index) for index in row.values if index != 0]), axis=1).values
X_test_sparse = vectorizer.fit_transform(corpus)

In [49]:
X_train_sparse.shape, X_test_sparse.shape

((253561, 48371), (82797, 48371))

Еще один плюс использования разреженных матриц в том, что для них имеются специальные реализации как матричных операций, так и алгоритмов машинного обучения, что подчас позволяет ощутимо ускорить операции за счет особенностей структуры данных. Это касается и логистической регрессии. Вот теперь у нас все готово для построения нашей первой модели.

### 2. Построение первой модели

Итак, у нас есть алгоритм и данные для него, построим нашу первую модель, воспользовавшись релизацией [логистической регрессии](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) из пакета `sklearn` с параметрами по умолчанию. Первые 90% данных будем использовать для обучения (обучающая выборка отсортирована по времени), а оставшиеся 10% для проверки качества (validation). 

**Напишите простую функцию, которая будет возвращать качество модели на отложенной выборке, и обучите наш первый классификатор**.

In [50]:
from sklearn.model_selection import train_test_split

def get_auc_lr_valid(X, y, C=1.0, ratio = 0.9, seed=17):
    '''
    X, y – выборка
    ratio – в каком отношении поделить выборку
    C, seed – коэф-т регуляризации и random_state 
              логистической регрессии
    '''
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=(1 - ratio), stratify = None, shuffle=False)
    est = LogisticRegression(C=C, random_state=seed)
    est.fit(X_train, y_train)
    return roc_auc_score(y_test, est.predict_proba(X_test)[:, 1])

**Посмотрите, какой получился ROC AUC на отложенной выборке.**

In [51]:
get_auc_lr_valid(X_train_sparse, y_train)

0.92357815290754763

Будем считать эту модель нашей первой отправной точкой (baseline). Для построения модели для прогноза на тестовой выборке **необходимо обучить модель заново уже на всей обучающей выборке** (пока наша модель обучалась лишь на части данных), что повысит ее обобщающую способность:

In [52]:
# функция для записи прогнозов в файл
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index = np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

**Обучите модель на всей выборке, сделайте прогноз для тестовой выборки и сделайте посылку в соревновании**.

In [53]:
est = LogisticRegression(random_state=17, n_jobs=-1)
est.fit(X_train_sparse, y_train)
y_pred = est.predict_proba(X_test_sparse)[:, 1]

c:\program files (x86)\python35-32\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


In [54]:
write_to_submission_file(y_pred, './../../data/alice_result1.csv')

Если вы выполните эти действия и загрузите ответ на [странице](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования, то воспроизведете первый бенчмарк "Logit".

### 3. Улучшение модели, построение новых признаков

Создайте такой признак, который будет представлять собой число вида ГГГГММ от той даты, когда проходила сессия, например 201407 -- 2014 год и 7 месяц. Таким образом, мы будем учитывать помесячный [линейный тренд](http://people.duke.edu/~rnau/411trend.htm) за весь период предоставленных данных.

In [55]:
new_train = pd.DataFrame(index=train_df.index)
new_test = pd.DataFrame(index=test_df.index)

In [56]:
def scale_feature(train, test, feat_name):
    scaler = StandardScaler()
    scaler.fit(train[feat_name].astype('float64').values.reshape(-1, 1))
    train[feat_name + '_s'] = scaler.transform(train[feat_name].astype('float64').values.reshape(-1, 1))
    test[feat_name + '_s'] = scaler.transform(test[feat_name].astype('float64').values.reshape(-1, 1))    

Добавьте новый признак, предварительно отмасштабировав его с помощью `StandardScaler`, и снова посчитайте ROC AUC на отложенной выборке.

In [57]:
def transform_sess_start(ts):
    return ts.year*100 + ts.month

new_train['year_month'] = train_df['time1'].apply(transform_sess_start)
new_test['year_month'] = test_df['time1'].apply(transform_sess_start)

In [58]:
scaler = StandardScaler()
scaler.fit(new_train['year_month'].astype('float64').values.reshape(-1, 1))
new_train['year_month_s'] = scaler.transform(new_train['year_month'].astype('float64').values.reshape(-1, 1))
new_test['year_month_s'] = scaler.transform(new_test['year_month'].astype('float64').values.reshape(-1, 1))

In [59]:
new_test.head()

,year_month,year_month_s
session_id,,
1,201410,0.822948
2,201407,0.752287
3,201412,0.870055
4,201411,0.846501
5,201405,0.705179


**Добавьте два новых признака: start_hour и morning.**

Признак `start_hour` – это час в который началась сессия (от 0 до 23), а бинарный признак `morning` равен 1, если сессия началась утром и 0, если сессия началась позже (будем считать, что утро это если `start_hour равен` 11 или меньше).

**Посчитйте ROC AUC на отложенной выборке для выборки с:**
- сайтами, `start_month` и `start_hour`
- сайтами, `start_month` и `morning`
- сайтами, `start_month`, `start_hour` и `morning`

In [60]:
def get_hour(ts):
    return ts.hour

def get_morning(ts):
    if ts.hour <= 11:
        return 1
    else:
        return 0
    
new_train['start_hour'] = train_df['time1'].apply(get_hour)
new_test['start_hour'] = test_df['time1'].apply(get_hour)
new_train['morning'] = train_df['time1'].apply(get_morning)
new_test['morning'] = test_df['time1'].apply(get_morning)

In [61]:
hour_scaler = StandardScaler()
hour_scaler.fit(new_train['start_hour'].astype('float64').values.reshape(-1, 1))
new_train['start_hour_s'] = hour_scaler.transform(new_train['start_hour'].astype('float64').values.reshape(-1, 1))
new_test['start_hour_s'] = hour_scaler.transform(new_test['start_hour'].astype('float64').values.reshape(-1, 1))

In [62]:
def get_sess_length(row):
    dropped_row = row.dropna()
    length_ts = dropped_row.iloc[-1] - dropped_row.iloc[-(2%len(dropped_row))]
    return length_ts/np.timedelta64(1, 's')

new_train['sess_length'] = train_df[times].apply(get_sess_length, axis=1)
new_test['sess_length'] = test_df[times].apply(get_sess_length, axis=1)

In [63]:
scale_feature(new_train, new_test, 'sess_length')

In [64]:
def get_site_count(row):
    dropped_row = row.dropna()
    return len(dropped_row)

new_train['site_count'] = train_df[times].apply(get_site_count, axis=1)
new_test['site_count'] = test_df[times].apply(get_site_count, axis=1)

In [69]:
scale_feature(new_train, new_test, 'site_count')

In [70]:
new_test.head()

,year_month,year_month_s,start_hour,morning,start_hour_s,sess_length,sess_length_s,site_count,site_count_s
session_id,,,,,,,,,
1,201410,0.822948,11,1,-0.407823,0.0,-0.203780,10,0.278784
2,201407,0.752287,11,1,-0.407823,23.0,0.029935,10,0.278784
3,201412,0.870055,15,0,0.858234,3.0,-0.173295,10,0.278784
4,201411,0.846501,10,1,-0.724338,0.0,-0.203780,10,0.278784
5,201405,0.705179,15,0,0.858234,4.0,-0.163134,10,0.278784


In [87]:
# X_train_2feat_start_hour = csr_matrix(hstack([X_train_sparse, 
#                                               new_train[['year_month_s', 'start_hour']].values]))

# X_train_2feat_morning = csr_matrix(hstack([X_train_sparse, 
#                                               new_train[['year_month_s', 'morning']].values]))

X_train_3feat = csr_matrix(hstack([X_train_sparse, 
                                   new_train[['year_month_s', 'start_hour_s', 'morning']].values]))

X_train_4feat = csr_matrix(hstack([X_train_sparse, 
                                   new_train[['year_month_s', 'start_hour_s', 'morning', 'sess_length_s']].values]))

In [72]:
# print(get_auc_lr_valid(X_train_2feat_morning, y_train))
# print(get_auc_lr_valid(X_train_2feat_start_hour, y_train))
# print(get_auc_lr_valid(X_train_3feat, y_train))

In [88]:
X_test_3feat = csr_matrix(hstack([X_test_sparse, 
                                  new_test[['year_month_s', 'start_hour', 'morning']].values]))

X_test_4feat = csr_matrix(hstack([X_test_sparse, 
                                  new_test[['year_month_s', 'start_hour', 'morning', 'sess_length_s']].values]))

In [149]:
est = LogisticRegression(random_state=17, n_jobs=-1, C=0.1)
est.fit(X_train_3feat, y_train)
y_pred = est.predict_proba(X_test_3feat)[:, 1]
write_to_submission_file(y_pred, './../../data/alice_result2.csv')

c:\program files (x86)\python35-32\lib\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


### 4. Подбор коэффицициента регуляризации

Итак, мы ввели признаки, которые улучшают качество нашей модели по сравнению с первым бейслайном. Можем ли мы добиться большего значения метрики? После того, как мы сформировали обучающую и тестовую выборки, почти всегда имеет смысл подобрать оптимальные гиперпараметры -- характеристики модели, которые не изменяются во время обучения. Например, на 3 неделе вы проходили решающие деревья, глубина дерева это гиперпараметр, а признак, по которому происходит ветвление и его значение -- нет. В используемой нами логистической регрессии веса каждого признака изменяются и во время обучения находится их оптимальные значения, а коэффициент регуляризации остается постоянным. Это тот гиперпараметр, который мы сейчас будем оптимизировать.

Посчитайте качество на отложенной выборке с коэффициентом регуляризации, который по умолчанию `C=1`:

In [80]:
get_auc_lr_valid(X_train_4feat, y_train)

0.95747123730027317

Постараемся побить этот результат за счет оптимизации коэффициента регуляризации. Возьмем набор возможных значений C и для каждого из них посчитаем значение метрики на отложенной выборке.

Найдите `C` из `np.logspace(-3, 1, 10)`, при котором ROC AUC на отложенной выборке максимален. 

In [89]:
for C in np.logspace(-3, 1, 10):
    print('C: ', C, 'ROC_AUC: ', get_auc_lr_valid(X_train_3feat, y_train, C=C))

C:  0.001 ROC_AUC:  0.823932622161
C:  0.00278255940221 ROC_AUC:  0.8934026784
C:  0.00774263682681 ROC_AUC:  0.939785739473
C:  0.0215443469003 ROC_AUC:  0.956403466982
C:  0.0599484250319 ROC_AUC:  0.960467892772
C:  0.16681005372 ROC_AUC:  0.961099331698
C:  0.464158883361 ROC_AUC:  0.960268634015
C:  1.29154966501 ROC_AUC:  0.958622786869
C:  3.5938136638 ROC_AUC:  0.955717231897
C:  10.0 ROC_AUC:  0.951314820987


Наконец, обучите модель с найденным оптимальным значением коэффициента регуляризации и с построенными признаками `start_hour`, `start_month` и `morning`. Если вы все сделали правильно и загрузите это решение, то повторите второй бенчмарк соревнования.

In [90]:
est = LogisticRegression(random_state=17, C=0.464158883361)
est.fit(X_train_3feat, y_train)
y_pred = est.predict_proba(X_test_3feat)[:, 1]
write_to_submission_file(y_pred, './../../data/alice_result3.csv')
print(get_auc_lr_valid(X_train_3feat, y_train, C=0.464158883361))

0.960269388783


0.96215593145